# San Francisco Rental Prices Dashboard

In this notebook, you will compile the visualizations from the previous analysis into functions that can be used for a Panel dashboard.

In [5]:
# initial imports
import os
import pandas as pd
import matplotlib.pyplot as plt
import hvplot.pandas
import panel as pn
import plotly.express as px
from pathlib import Path
from dotenv import load_dotenv

# Initialize the Panel Extensions (for Plotly)
pn.extension('plotly')

In [6]:
# Read the Mapbox API key
load_dotenv()
mapbox_token = os.getenv("MAPBOX")

# Set the Mapbox API
px.set_mapbox_access_token(mapbox_token)

# Import Data

In [7]:
# Import the CSVs to Pandas DataFrames
file_path = Path("Data/sfo_neighborhoods_census_data.csv")
sfo_data = pd.read_csv(file_path, index_col="year")

file_path = Path("Data/neighborhoods_coordinates.csv")
df_neighborhood_locations = pd.read_csv(file_path)

- - -

## Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. 

These functions will convert the plot object to a Panel pane.

Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.

Return a Panel pane object from each function that can be used to build the dashboard.

Note: Remove any `.show()` lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

In [146]:
# Define Panel Visualization Functions
def housing_units_per_year():
    """Housing Units Per Year."""    
    sum_housing_units_per_year = sfo_data[['housing_units']].groupby('year').sum()    
    
#     sum_housing_units_per_year.plot(kind = 'bar')    
#     plt.title('Housing Units in san Francisco from 2010 to 2016') 
#     plt.xlabel('Year')
#     plt.ylabel('Housing Unit')
#     plt.ylim(15000000, 24000000)
    
#     fig_housing_units = plt.figure()    
    
#     plt.show()
#     plt.close(fig_housing_units)
#     return plt

    return sum_housing_units_per_year.hvplot.bar(
        title = 'Housing Units in san Francisco from 2010 to 2016', 
        x ='Year', 
        ylabel = 'Housing Unit', 
        rot = 90, 
        height=600, 
        width=1200)

def average_gross_rent():
    """Average Gross Rent in San Francisco Per Year."""    
    average_gross_rent_per_year = sfo_data[['gross_rent']].groupby('year').mean() 
    average_gross_rent_per_year.sort_values(by='year', inplace=True)  

    #     average_gross_rent_per_year.plot(kind = 'line');    
    #     plt.title('Average Gross Rent in San Francisco')
    #     plt.xlabel('Year')
    #     plt.ylabel('Gross rent')    
    #     return plt

    return average_gross_rent_per_year.hvplot.line(
        title = 'Average Gross Rent in San Francisco', 
        x ='Year', 
        ylabel = 'Gross rent', 
        rot = 90, 
        height=600, 
        width=1200)


def average_sales_price():
    """Average Sales Price Per Year."""
    average_sales_price_per_year = sfo_data[['sale_price_sqr_foot']].groupby('year').mean()
    average_sales_price_per_year.sort_values(by='year', inplace=True)   

    #     average_sales_price_per_year.plot(kind = 'line');    
    #     plt.title('Average Sales Price per Square Foot in San Francisco')
    #     plt.xlabel('Year')
    #     plt.ylabel('Sales price sqr foot')    
    #     return plt

    return average_sales_price_per_year.hvplot.line(
        title = 'Average Sales Price per Square Foot in San Francisco', 
        x ='Year', 
        ylabel = 'Sales price sqr foot', 
        rot = 90, 
        height=600, 
        width=1200)

def average_price_by_neighborhood():
    """Average Prices by Neighborhood."""
    average_price_per_neighborhood = sfo_data[['neighborhood', 'sale_price_sqr_foot']].groupby('neighborhood').mean()
    average_price_per_neighborhood.sort_values(by='neighborhood', inplace=True) 

    #     average_price_per_neighborhood.plot(kind = 'bar');    
    #     plt.title('Average Prices by Neighborhood')
    #     plt.xlabel('Neighborhood')
    #     plt.ylabel('Sales price sqr foot')    
    #     return plt

    return average_price_per_neighborhood.hvplot.bar(
        title = 'Average Prices by Neighborhood', 
        x ='neighborhood', 
        ylabel = 'sale price per sqr foot', 
        rot = 90, 
        height=600, 
        width=1200)

def most_expensive_neighborhoods_data_frame():
    """Top 10 Most Expensive Neighborhoods."""
    most_expensive_neighborhoods = sfo_data[['neighborhood', 'sale_price_sqr_foot', 'housing_units', 'gross_rent']].groupby('neighborhood').max()
    most_expensive_neighborhoods.reset_index(inplace=True)
    most_expensive_neighborhoods.sort_values(by='sale_price_sqr_foot', inplace=True)        
#     most_expensive_neighborhoods = most_expensive_neighborhoods[-10:]    
    return most_expensive_neighborhoods


def top_most_expensive_neighborhoods():
    """Top 10 Most Expensive Neighborhoods."""
    most_expensive_neighborhoods = most_expensive_neighborhoods_data_frame()
    most_expensive_neighborhoods = most_expensive_neighborhoods[['neighborhood', 'sale_price_sqr_foot']]
    most_expensive_neighborhoods.sort_values(by='sale_price_sqr_foot', inplace=True)        
    most_expensive_neighborhoods = most_expensive_neighborhoods[-10:]
    
    return most_expensive_neighborhoods.hvplot.line(
        title = 'Top 10 expensive neighborhoods', 
        x ='neighborhood', 
        y= 'sale_price_sqr_foot',
        ylabel = 'sale price per sqr foot', 
        rot = 90, 
        height=600, 
        width=1200)    

def parallel_coordinates():
    """Parallel Coordinates Plot."""    
    most_expensive_df = most_expensive_neighborhoods_data_frame()
    
    return px.parallel_coordinates(
        most_expensive_df,
        dimensions=['neighborhood', 'sale_price_sqr_foot', 'housing_units', 'gross_rent'],
        color="sale_price_sqr_foot",
        color_continuous_scale=px.colors.sequential.Inferno,
        labels={
            "neighborhood": "neighborhood",
            "sale_price_sqr_foot": "sale price sqr foot",
            "housing_units": "housing units",
            "gross_rent": "gross rent",
        },
        width=1200
    )

def parallel_categories():
    """Parallel Categories Plot."""
    most_expensive_df = most_expensive_neighborhoods_data_frame()
    
    return px.parallel_categories(
        most_expensive_df,
        dimensions=['neighborhood', 'sale_price_sqr_foot', 'housing_units', 'gross_rent'],
        color="sale_price_sqr_foot",
        color_continuous_scale=px.colors.sequential.Inferno,
        labels={
            "neighborhood": "neighborhood",
            "sale_price_sqr_foot": "sale price sqr foot",
            "housing_units": "housing units",
            "gross_rent": "gross rent",
        },
        width=1200
    )
    

def neighborhood_map():
    """Neighborhood Map"""
    # Load neighborhoods coordinates data
    file_path = Path("Data/neighborhoods_coordinates.csv")
    df_neighborhood_locations = pd.read_csv(file_path)
    df_neighborhood_locations.head()
    
    # Calculate the mean values for each neighborhood
    mean_df = (
        sfo_data[['neighborhood', 'sale_price_sqr_foot', 'housing_units', 'gross_rent']]
        .groupby('neighborhood')
        .mean()
    ).reset_index()

    mean_df.rename(columns={'neighborhood':'Neighborhood'}, inplace=True)
    mean_df.head()
    
    # Join the average values with the neighborhood locations
    neighborhood_average_values_df = df_neighborhood_locations.merge(mean_df, left_on='Neighborhood', right_on='Neighborhood')
    neighborhood_average_values_df.head()
    
    # Create a scatter mapbox to analyze neighborhood info
    map = px.scatter_mapbox(
        neighborhood_average_values_df,
        lat="Lat",
        lon="Lon",
        size="sale_price_sqr_foot",
        color="Neighborhood",
        zoom=8
    )

    # Display the map
    map.show()
    
    return map

## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [147]:
# YOUR CODE HERE!
tabs = pn.Tabs(
    ("housing units per year", housing_units_per_year),
    ("average gross rent", average_gross_rent),
    ("average sales price", average_sales_price),
    ("average price by neighborhood", average_price_by_neighborhood),
    ("top most expensive neighborhoods", top_most_expensive_neighborhoods),
    ("parallel categories", parallel_categories),
    ("parallel coordinates", parallel_coordinates)
)
tabs

Tabs
    [0] Column
        [0] Column()
        [1] Row
            [0] HoloViews(Bars, name='interactive11413')
    [1] Column
        [0] Column()
        [1] Row
            [0] HoloViews(Curve, name='interactive11495')
    [2] Column
        [0] Column()
        [1] Row
            [0] HoloViews(Curve, name='interactive11577')
    [3] Column
        [0] Column()
        [1] Row
            [0] HoloViews(Bars, name='interactive11659')
    [4] Column
        [0] Column()
        [1] Row
            [0] HoloViews(Curve, name='interactive11741')
    [5] Column
        [0] Column()
        [1] Row
            [0] Plotly(Figure, name='interactive11824')
    [6] Column
        [0] Column()
        [1] Row
            [0] Plotly(Figure, name='interactive11830')

## Serve the Panel Dashboard

In [104]:
# panel.servable()